In [ ]:
!python main.py

In [2]:
!pip install bs4

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     -------- ------------------------------ 30.7/143.0 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 143.0/143.0 kB 2.8 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1266 sha256=4b880058f62b0385d64cce74e71ff7809f500136120eaed5b216da9733164f28
  Stored in directory: c:\users\clo36\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [4]:
from scholar.post_maker import make_all

make_all(overwrite=True)

Markdown file created successfully: SimBIG: mock challenge for a forward modeling approach to galaxy clustering
Markdown file created successfully: Albatross: A scalable simulation-based inference pipeline for analysing stellar streams in the Milky Way
Markdown file created successfully: Peregrine: Sequential simulation-based inference for gravitational wave signals
Markdown file created successfully: Amortized Bayesian inference on generative dynamical network models of epilepsy using deep neural density estimators
Markdown file created successfully: Simulation-based Inference for Model Parameterization on Analog Neuromorphic Hardware
Markdown file created successfully: Investigating the turbulent hot gas in X-COP galaxy clusters
Markdown file created successfully: Constraining the X-ray heating and reionization using 21-cm power spectra with Marginal Neural Ratio Estimation
Markdown file created successfully: Modelling capture efficiency of single cell RNA-sequencing data improves in

# Archive

## Add new column called arxiv_term in SQLite

In [ ]:
import sqlite3

# Reset the new columns
# with sqlite3.connect('scholar/paper.db') as conn:
#     c = conn.cursor()
#     c.execute('ALTER TABLE paper DROP COLUMN arxiv_term_short;')
#     c.execute('ALTER TABLE paper DROP COLUMN arxiv_term_long;')
#     conn.commit()

with sqlite3.connect("scholar/paper.db") as conn:
    c = conn.cursor()
    c.execute("ALTER TABLE paper ADD COLUMN arxiv_group_tag VARCHAR(255);")
    c.execute("ALTER TABLE paper ADD COLUMN arxiv_category_tag VARCHAR(255);")
    conn.commit()

## Patch category and group tag to database

In [ ]:
from time import sleep
from scholar.database import Paper
from scholar.api import ARXIV_GROUP, ARXIV_CATEGORY, get_arxiv_category

p = Paper.select().where(Paper.arxiv_group_tag == None)

for paper in p.iterator():
    print(paper.title)
    try:
        category_tag = get_arxiv_category(paper.title)
    except AttributeError:
        print(f"{paper.title} not found.")
        continue

    # Make group tag and push to database
    group_tag = category_tag.split(".")[0]
    if group_tag in ARXIV_GROUP:
        paper.arxiv_group_tag = group_tag

    if category_tag in ARXIV_CATEGORY:
        paper.arxiv_category_tag = category_tag
    paper.save()
    sleep(0.5)  # arxiv.org has a rate limit of 3 requests per second